In [ ]:
import numpy as np
import warnings

from scipy.stats import expon, norm, uniform

from envs.vases_grid import VasesGrid, VasesEnvState #, print_state, str_to_state, state_to_str
from envs.utils import unique_perm, zeros_with_ones, printoptions
from envs.vases_spec import VasesEnvState2x3V2D3, VasesEnvSpec2x3V2D3, VasesEnvState2x3Broken, VasesEnvSpec2x3Broken

from value_iter_and_policy import vi_boltzmann_deterministic, vi_rational_deterministic
from occupancy_measure import compute_d_last_step_deterministic

from sampling_one_s_mceirl import policy_walk_last_state_prob, log_last_step_om
from test import forward_rl

In [ ]:
env2x3v2d3 = VasesGrid(VasesEnvSpec2x3V2D3())
r_spec = np.array([0, 0, 1, 0, 0, 0])
s_current = np.zeros(env2x3v2d3.nS)
s_current[1582] = 1
env2x3v2d3.print_state(env2x3v2d3.get_state_from_num(1582))
p_0=np.zeros(env2x3v2d3.nS)
p_0[env2x3v2d3.get_num_from_state(env2x3v2d3.init_state)] = 1

In [ ]:
warnings.filterwarnings('error')

# Negative exponential prior with mode at r_spec
# r_prior = neg_exp_distr(mode=r_spec, scale=.2); step_size=.0003

# Gaussian prior with mean at r_spec
# r_prior = norm(loc=r_spec, scale=.05); step_size=.01

# Uniform prior centered at r_pec. Subtracting half scale such that center of the 
# distribution is at loc, as oppposed to the lowest end
# scale=2; loc=r_spec-.5*scale; r_prior = uniform(loc=loc, scale=scale); step_size=.01

# Uniform prior centered at 0. Subtracting half scale such that center of the 
# distribution is at loc, as oppposed to the lowest end

r_samples_list = []
#for step_size in [0.1, .0001, .00001, .000001]:
for i in range(10):
    step_size=0.1
    scale=2; 
    loc=np.zeros_like(r_spec)-.5*scale; 
    r_prior = uniform(loc=loc, scale=scale); 
    horizon = 70
    temp=1
    n_samples=2000
    
    r_samples = policy_walk_last_state_prob(env2x3v2d3, s_current, p_0, horizon, temp,
                            n_samples, step_size, r_prior)
    r_samples_list.append(r_samples)
    with printoptions(precision=4, suppress=True):
        res = np.mean(r_samples[250::], axis=0)
        print(res)
        print(np.var(r_samples[250::], axis=0))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

'''
Order of features:
- Number of broken vases
- Number of vases on tables
- Number of tablecloths on tables
- Number of tablecloths on floors
- Number of vases on desks
- Number of tablecloths on desks
'''
with printoptions(precision=4, suppress=True):
    res = np.mean(r_samples[250::], axis=0)
    print(res)
    plt.matshow(res.reshape(1,-1))
    plt.yticks([], [])
    plt.colorbar()

In [ ]:
forward_rl(env2x3v2d3, np.mean(r_samples[250::], axis=0), steps_printed=100)